In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from lambda_function import baseball_lambda

In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs - {today}"

In [4]:
session_id = '882432'

In [5]:
task = "Consider the first week of June 2019 - find 3 pitchers who's changeups were most similar to Justin Verlander's"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Plan'}}
----
Distance to neareast plan: 0.3361414670944214
Modifying nearest plan with User input
{'Plan': {'messages': [HumanMessage(content='Original Plan:\nRevised Plan:\n\n1. Use the `playerid_lookup` function to find Justin Verlander\'s MLBAM player ID. Pass in "Verlander" as the `last` argument and "Justin" as the `first` argument. From the output, note the `key_mlbam` field for Justin Verlander.\n\n2. Use the `statcast_pitcher` function to retrieve pitch-level statcast data for Justin Verlander for the first week of June 2019. Set `start_dt` to "2019-06-01", `end_dt` to "2019-06-07", and `player_id` to Justin Verlander\'s MLBAM ID obtained in step 1. \n\n3. From the output of `statcast_pitcher`, filter the data to only include pitches where `pitch_type` is "CH" (changeup). Note the average values of `release_speed`, `release_spin`, `pfx_x`, and `pfx_z` for Verlander\'s changeups.\n\n4. Use the `statcast_pitcher` function to retrieve pitch-level statcast

In [6]:
task = "looks good"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Execute'}}
----


Python REPL can execute arbitrary code. Use with caution.


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "__arg1": "from pybaseball import playerid_lookup\\n\\nverlander_id = playerid_lookup(\'Verlander\', \'Justin\')[\'key_mlbam\'][0]\\nprint(verlander_id)"\n}', 'name': 'Python_REPL'}})]}}
----
{'action': {'messages': [FunctionMessage(content='Gathering player lookup table. This may take a moment.\n434378\n', name='Python_REPL')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "__arg1": "from pybaseball import statcast_pitcher\\n\\nverlander_data = statcast_pitcher(\'2019-06-01\', \'2019-06-07\', verlander_id)\\nverlander_changeups = verlander_data[verlander_data[\'pitch_type\'] == \'CH\']\\n\\nverlander_avg_release_speed = verlander_changeups[\'release_speed\'].mean()\\nverlander_avg_release_spin = verlander_changeups[\'release_spin_rate\'].mean()\\nverlander_avg_pfx_x = verlander_changeups[\'pfx_x\'].mean()\\nverlander_avg_pf

  0%|          | 0/7 [00:00<?, ?it/s]

{'agent': {'messages': [AIMessage(content='It seems like the `statcast_pitcher` function requires a player ID and cannot be used to retrieve data for all players. I will need to modify the plan to use the `statcast` function instead, which retrieves data for all players.\n\nModified Plan:\n\n1. Use the `playerid_lookup` function to find Justin Verlander\'s MLBAM player ID. Pass in "Verlander" as the `last` argument and "Justin" as the `first` argument. From the output, note the `key_mlbam` field for Justin Verlander.\n\n2. Use the `statcast_pitcher` function to retrieve pitch-level statcast data for Justin Verlander for the first week of June 2019. Set `start_dt` to "2019-06-01", `end_dt` to "2019-06-07", and `player_id` to Justin Verlander\'s MLBAM ID obtained in step 1. \n\n3. From the output of `statcast_pitcher`, filter the data to only include pitches where `pitch_type` is "CH" (changeup). Note the average values of `release_speed`, `release_spin`, `pfx_x`, and `pfx_z` for Verland

100%|██████████| 7/7 [00:03<00:00,  1.99it/s]


{'action': {'messages': [FunctionMessage(content='This is a large query, it may take a moment to complete\n                   release_speed  release_spin_rate     pfx_x     pfx_z\nplayer_name                                                            \nAdams, Austin               90.3             2006.0     -1.15      0.31\nAlbers, Matt           82.533333        1467.333333 -1.096667      0.49\nAlcantara, Sandy       88.354545        1957.363636     -1.32  0.431818\nAlcántara, Victor           88.7             1852.0     -1.41      0.65\nAlexander, Scott            87.7             1711.5      1.06     0.185\n...                          ...                ...       ...       ...\nWood, Hunter            85.54375          1656.5625 -0.834375  1.191875\nWoodruff, Brandon      87.222222        1757.166667 -1.253611  0.656389\nYarbrough, Ryan        79.113636        2028.954545     1.445      0.69\nYnoa, Gabriel          85.606667        1909.566667    -1.429  0.703333\ndeGrom, Jacob    

/Users/rosclay/Documents/VSCode/baseball-langchain-agents/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "answer": "The three pitchers whose changeups were most similar to Justin Verlander\'s during the first week of June 2019 are Adam Warren, Brandon Woodruff, and Taylor Williams.",\n  "python_code": "from pybaseball import playerid_lookup\\n\\nverlander_id = playerid_lookup(\'Verlander\', \'Justin\')[\'key_mlbam\'][0]\\n\\nfrom pybaseball import statcast_pitcher\\n\\nverlander_data = statcast_pitcher(\'2019-06-01\', \'2019-06-07\', verlander_id)\\nverlander_changeups = verlander_data[verlander_data[\'pitch_type\'] == \'CH\']\\n\\nverlander_avg_release_speed = verlander_changeups[\'release_speed\'].mean()\\nverlander_avg_release_spin = verlander_changeups[\'release_spin_rate\'].mean()\\nverlander_avg_pfx_x = verlander_changeups[\'pfx_x\'].mean()\\nverlander_avg_pfx_z = verlander_changeups[\'pfx_z\'].mean()\\n\\nfrom pybaseball import statcast\\n\\nall_players_data = statcast(\'2019-06-01

In [7]:
task = "yes"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Memorize'}}
----
{'Memorize': {'messages': [HumanMessage(content='Thank you, task has been written to memory', name='Memorizer')]}}
----
Thank you, task has been written to memory
